# Retrieval-Augmented Generation (RAG)

In [26]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace, ModelSettings

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [2]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [3]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [4]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [5]:
calorie_lookup_tool('bananas')

TypeError: 'FunctionTool' object is not callable

In [6]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
    tools=[calorie_lookup_tool],
)

In [10]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a pizza slice and an apple? Also give calories per 100g",
    )
    print(result.final_output)

Here’s what we have per 100 g:
- Apple: 52 kcal per 100 g
- Pizza (varies by type):
  - Thin crust: 261 kcal per 100 g
  - New York style: 169 kcal per 100 g
  - Generic pizza: 267 kcal per 100 g

To give total calories for “a pizza slice and an apple,” I need the slice weight (in g) and the apple weight (in g). If you want a rough example, using 120 g per slice and a 182 g apple:
- Apple: 182 g × 52/100 ≈ 95 kcal
- Pizza slice (choose type):
  - Thin crust: 120 g × 261/100 ≈ 313 kcal
  - New York style: 120 g × 169/100 ≈ 203 kcal
  - Generic: 120 g × 267/100 ≈ 320 kcal
Total would be Apple ≈ 95 kcal + Slice ≈ 203–313 kcal depending on type. Need your exact weights or preferred pizza type to be precise.


In [29]:
# Test the setup with sample queries
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

In [53]:
@function_tool
def qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to answer questions about nutrition during pregnancy.

    Args:
        query: question about pregnancy.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the answer to the question about pregnancy.
    """
    print("=== Query: ===", query)
    results = nutrition_qna.query(query_texts=[query], n_results=3)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [54]:
nutrition_agent = Agent(
    name="Nutrition Assist for pregnancy",
    instructions="""
    You are a helpful nutrition assistant giving out nutrition advice for pregnancies.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[qna_tool, calorie_lookup_tool],

)

In [55]:
with trace("Nutrition Qna with RAG"):
    result = await Runner.run(
        nutrition_agent,
        "What are the nutrients that pregnant women need? how much is needed daily for each nutrient?  what are the best foods to eat",
    )
    print(result.final_output)

=== Query: === What nutrients are essential during pregnancy and their daily requirements, plus best foods to eat?
Here’s a concise guide.

Essential nutrients and daily amounts (pregnancy):
- Folate (folic acid): ~600 mcg dietary folate equivalents (DFE) daily
  - Best foods: leafy greens, legumes, fortified cereals, citrus, eggs
- Iron: ~27 mg/day
  - Best foods: lean red meat, poultry, fish, beans, fortified cereals, spinach; pair with vitamin C source to enhance absorption
- Calcium: ~1000 mg/day
  - Best foods: dairy or fortified plant milks/yogurts, cheese, leafy greens (not oxalate-heavy), sardines with bones
- Vitamin D: ~600 IU/day
  - Best foods: fortified dairy/plant milks, fatty fish (low-mercury), egg yolks; sun exposure as appropriate
- Iodine: ~220 mcg/day
  - Best foods: iodized salt, dairy, seafood
- Protein: ~71 g/day
  - Best foods: lean meats, eggs, dairy, beans, lentils, tofu, nuts/seeds
- Omega-3 DHA: ~200–300 mg/day
  - Best foods: fatty fish (salmon, sardines) 2